In [1]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os
import plotly.express as px

# local imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.calibration_processing import two_point_calibration, process_bottle

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

df_p_413 = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "picarro", "DWD_Picarro_G2301_413.parquet")) 
df_p_529 = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "picarro", "ICOS_Picarro_G2401_529.parquet")) 

In [ ]:
df_p_413.head(3).collect()

In [3]:
def process_two_point_picarro_calibration(df, low_start_date, low_end_date, high_start_date, high_end_date):

    df_p_400 = df.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
                    pl.col("CO2_dry"), 
                    pl.col("h2o_reported")) \
                    .sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(low_start_date, low_end_date)) \
                    .collect()


    df_p_600 = df.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
                    pl.col("CO2_dry"), 
                    pl.col("h2o_reported")) \
                    .sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(high_start_date, high_end_date)) \
                    .collect()


    measured_values = [None, None]
    true_values = [427.38, 610.95]

    # 400 ppm
    data = df_p_400.select(pl.col("CO2_dry")).to_series().to_list()
    measured_values[0] = process_bottle(data=data, ignore_len=True)

    # 600 ppm
    data = df_p_600.select(pl.col("CO2_dry")).to_series().to_list()
    measured_values[1] = process_bottle(data=data, ignore_len=True)

    print(measured_values)

    picarro_slope, picarro_intercept = two_point_calibration(measured_values, true_values)
    print(picarro_slope, picarro_intercept)

    #plots
    fig = px.line(df_p_400, x="creation_timestamp", y="h2o_reported")
    fig.show()

    fig = px.line(df_p_400, x="creation_timestamp", y="CO2_dry")
    fig.show()

    fig = px.line(df_p_600, x="creation_timestamp", y="h2o_reported")
    fig.show()

    fig = px.line(df_p_600, x="creation_timestamp", y="CO2_dry")
    fig.show()

In [ ]:
# 23.10.2023

low_start_date = datetime(2023, 10, 23, 13, 31, 30).replace(tzinfo=timezone.utc)
low_end_date = datetime(2023, 10, 23, 14, 6, 0).replace(tzinfo=timezone.utc)

high_start_date = datetime(2023, 10, 23, 13, 6, 0).replace(tzinfo=timezone.utc)
high_end_date = datetime(2023, 10, 23, 13, 31, 0).replace(tzinfo=timezone.utc)

process_two_point_picarro_calibration(df_p_413, low_start_date, low_end_date, high_start_date, high_end_date)

In [ ]:
# 18.12.2023

low_start_date = datetime(2023, 12, 18, 14, 33, 0).replace(tzinfo=timezone.utc)
low_end_date = datetime(2023, 12, 18, 15, 2, 0).replace(tzinfo=timezone.utc)

high_start_date = datetime(2023, 12, 18, 15, 4, 0).replace(tzinfo=timezone.utc)
high_end_date = datetime(2023, 12, 18, 15, 32, 0).replace(tzinfo=timezone.utc)

process_two_point_picarro_calibration(df_p_413, low_start_date, low_end_date, high_start_date, high_end_date)

In [ ]:
# 04.09.2024: DWD Picarro

low_start_date = datetime(2024, 9, 4, 13, 00, 0).replace(tzinfo=timezone.utc)
low_end_date = datetime(2024, 9, 4, 13, 30, 0).replace(tzinfo=timezone.utc)

high_start_date = datetime(2024, 9, 4, 13, 31, 0).replace(tzinfo=timezone.utc)
high_end_date = datetime(2024, 9, 4, 14, 00, 0).replace(tzinfo=timezone.utc)

process_two_point_picarro_calibration(df_p_413, low_start_date, low_end_date, high_start_date, high_end_date)

In [ ]:
# 23.09.2024: ICOS Picarro

low_start_date = datetime(2024, 9, 23, 12, 19, 0).replace(tzinfo=timezone.utc)
low_end_date = datetime(2024, 9, 23, 12, 52, 0).replace(tzinfo=timezone.utc)

high_start_date = datetime(2024, 9, 23, 12, 54, 0).replace(tzinfo=timezone.utc)
high_end_date = datetime(2024, 9, 23, 13, 22, 0).replace(tzinfo=timezone.utc)

process_two_point_picarro_calibration(df_p_529, low_start_date, low_end_date, high_start_date, high_end_date)

[424.552779525, 607.02218673]
1.0060316565492182 0.2664639218892262


In [5]:
# 21.11.2024: ICOS Picarro

low_start_date = datetime(2024, 11, 21, 12, 26, 0).replace(tzinfo=timezone.utc)
low_end_date = datetime(2024, 11, 21, 12, 53, 30).replace(tzinfo=timezone.utc)

high_start_date = datetime(2024,11, 21, 12, 55, 0).replace(tzinfo=timezone.utc)
high_end_date = datetime(2024, 11, 21, 13, 23, 0).replace(tzinfo=timezone.utc)

process_two_point_picarro_calibration(df_p_529, low_start_date, low_end_date, high_start_date, high_end_date)

[424.61837593, 607.10035397]
1.0059623529495145 0.22989944385568606
